# Exam Timetabling with Morning and Afternoon Slots using OR-Tools

This notebook models exams scheduled over multiple days with morning and afternoon slots.

Constraints:
- No student can have two exams in the same slot (same day & same slot)
- No more than 2 exams per slot (day + morning/afternoon)

Outputs the exam schedule showing day and slot assignment.

In [1]:
# Install OR-Tools if you haven't yet
# !pip install ortools

In [2]:
from ortools.sat.python import cp_model

ImportError: dlopen(/Users/edwardbrady/Library/Python/3.9/lib/python/site-packages/ortools/sat/python/cp_model_helper.so, 0x0002): Library not loaded: /Users/corentinl/work/stable/temp_python3.9/lib/libscip.9.2.dylib
  Referenced from: <07FF885B-3BED-30C4-9BE0-B2A889DC2E5D> /Users/edwardbrady/Library/Python/3.9/lib/python/site-packages/ortools/sat/python/cp_model_helper.so
  Reason: tried: '/Users/corentinl/work/stable/temp_python3.9/lib/libscip.9.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/corentinl/work/stable/temp_python3.9/lib/libscip.9.2.dylib' (no such file), '/Users/corentinl/work/stable/temp_python3.9/lib/libscip.9.2.dylib' (no such file)

In [ ]:
# Sample data
exams = ['Math', 'Physics', 'Chemistry', 'History', 'Geography', 'PE', 'Music']

# Students and their exams
students = {
    'Alice': ['Math', 'Physics', 'Geography'],
    'Bob': ['Physics', 'Chemistry', 'Music'],
    'Charlie': ['Math', 'History', 'PE'],
    'Diana': ['Chemistry', 'History', 'Music'],
}

# Module leaders for info
module_leaders = {
    'Math': 'Dr. Smith',
    'Physics': 'Dr. Johnson',
    'Chemistry': 'Dr. Lee',
    'History': 'Dr. Patel',
    'Geography': 'Dr. Brown',
    'PE': 'Coach Carter',
    'Music': 'Ms. Green',
}

# Days and slots
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday']
slots = ['Morning', 'Afternoon']

In [ ]:
model = cp_model.CpModel()

num_days = len(days)
num_slots = len(slots)

# Variables: exam_day and exam_slot
exam_day = {}
exam_slot = {}
for exam in exams:
    exam_day[exam] = model.NewIntVar(0, num_days - 1, f'{exam}_day')
    exam_slot[exam] = model.NewIntVar(0, num_slots - 1, f'{exam}_slot')

In [ ]:
# Constraint 1: No student can have two exams at the same time (same day and same slot)
for student, stu_exams in students.items():
    for i in range(len(stu_exams)):
        for j in range(i + 1, len(stu_exams)):
            # Add constraint that exams i and j for the student can't be in the same day and slot
            model.AddBoolOr([
                exam_day[stu_exams[i]] != exam_day[stu_exams[j]],
                exam_slot[stu_exams[i]] != exam_slot[stu_exams[j]]
            ])

In [ ]:
# Constraint 2: No more than 2 exams per slot (day + morning/afternoon)
for d in range(num_days):
    for s in range(num_slots):
        exams_in_slot = []
        for exam in exams:
            is_in_slot = model.NewBoolVar(f'{exam}_day{d}_slot{s}')
            model.Add(exam_day[exam] == d).OnlyEnforceIf(is_in_slot)
            model.Add(exam_day[exam] != d).OnlyEnforceIf(is_in_slot.Not())
            model.Add(exam_slot[exam] == s).OnlyEnforceIf(is_in_slot)
            model.Add(exam_slot[exam] != s).OnlyEnforceIf(is_in_slot.Not())
            exams_in_slot.append(is_in_slot)
        model.Add(sum(exams_in_slot) <= 2)

In [ ]:
# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
    print("Exam Schedule:")
    for exam in exams:
        d = solver.Value(exam_day[exam])
        s = solver.Value(exam_slot[exam])
        leader = module_leaders.get(exam, 'Unknown')
        print(f" - {exam} (Leader: {leader}): {days[d]} {slots[s]}")
else:
    print("No solution found.")